In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm


In [2]:
datapath = "../data/ICPSR_04351/DS0002/04351-0002-Data.csv"

dump = pd.read_csv(datapath)


unemployment_datapath = "../data/ECONOMIC/UNRATE.csv"
unemployment = pd.read_csv(unemployment_datapath)

In [3]:
dump = dump[["MONTH","YEAR", "VICSEX", "VICRACE", "OFFSEX", "OFFRACE", "RELATION", "CIRCUM", "POPGROUP", "STCODE"]]

# use date for joining
dump['DATE'] = pd.to_datetime(dump[['YEAR', 'MONTH']].assign(DAY=1)).dt.date 

dump["DATE_and_STATE"] = dump["DATE"].astype(str) + "_" + dump["STCODE"].astype(str)

dump["COUNT"] = 1

dump["IN_FAMILY"] = dump["RELATION"].apply(lambda x: 1 if 1 <= 1 and x <= 16 else 0)
dump["KILLED_CHILD"] = dump["RELATION"].apply(lambda x: 1 if x in [7,8] else 0)
dump["IN_CITY"] = dump["POPGROUP"].apply(lambda x: 1 if x in [1, 2, 10, 11, 12, 13, 20] else 0)
dump["IN_SUBURB"] = dump["POPGROUP"].apply(lambda x: 1 if x in [3, 4, 5, 30, 40, 50] else 0)
dump["IN_RURAL"] = dump["POPGROUP"].apply(lambda x: 1 if x not in [1, 2, 10, 11, 12, 13, 20, 3, 4, 5, 30, 40, 50] else 0)

# drop unknown race and unknown sex for offender
dump = dump[dump['OFFSEX'] != 9]
dump = dump[dump['OFFRACE'] != 9]

# Collapse Race into white, non white; collapse sex into male
dump['WHITE_OFF'] = dump['OFFRACE'].apply(lambda x : 1 if x == 1 else 0)
dump['MALE_OFF'] = dump['OFFRACE'].apply(lambda x: 1 if x == 1 else 0)


# D_S_PANEL = dump.groupby("DATE_and_STATE")[["COUNT","IN_FAMILY", "KILLED_CHILD", "IN_CITY", "IN_SUBURB", "IN_RURAL"]].sum()


In [46]:
# new granuality of data

# Cole's group by


# what we want to keep
groupby_cols = ["DATE", "IN_FAMILY", "KILLED_CHILD", "IN_CITY", "WHITE_OFF", "MALE_OFF", "IN_RURAL"]
# groupby_cols = ["DATE", "IN_FAMILY", ]


drop_cols = [col for col in dump.columns if col not in groupby_cols] 
print("dropped cols" , drop_cols)

ts = dump.drop(columns=drop_cols, axis=1)


# maybe try not counting
ts = ts.groupby(groupby_cols)['DATE'].agg('count').to_frame("Counts").reset_index()
ts['DATE'] = pd.to_datetime(ts['DATE'])



start_date =  pd.to_datetime("1976-01-01")

# periods
ts['start_date'] = start_date
ts['PERIOD'] = (ts['DATE'].dt.year - ts['start_date'].dt.year) * 12 + (ts['DATE'].dt.month - ts['start_date'].dt.month)
ts['PERIOD_SQ'] = ts['PERIOD'] **2
ts['LOG_PERIOD'] = np.log(ts['PERIOD'] + 1)



print(ts)


dropped cols ['MONTH', 'YEAR', 'VICSEX', 'VICRACE', 'OFFSEX', 'OFFRACE', 'RELATION', 'CIRCUM', 'POPGROUP', 'STCODE', 'DATE_and_STATE', 'COUNT', 'IN_SUBURB']
           DATE  IN_FAMILY  KILLED_CHILD  IN_CITY  WHITE_OFF  MALE_OFF  \
0    1976-01-01          0             0        0          0         0   
1    1976-01-01          0             0        0          0         0   
2    1976-01-01          0             0        0          1         1   
3    1976-01-01          0             0        0          1         1   
4    1976-01-01          0             0        1          0         0   
...         ...        ...           ...      ...        ...       ...   
5999 2003-12-01          1             1        0          0         0   
6000 2003-12-01          1             1        0          1         1   
6001 2003-12-01          1             1        0          1         1   
6002 2003-12-01          1             1        1          0         0   
6003 2003-12-01          1   

In [47]:

# make both dates same type before merge

unemployment['DATE'] = pd.to_datetime(unemployment['DATE'])

# add year only column for inflation merge
ts['YEAR'] = ts['DATE'].dt.year


In [48]:
# inflation data
inflation_df = pd.read_csv('../data/ECONOMIC/FPCPITOTLZGUSA.csv')
inflation_df['DATE'] = pd.to_datetime(inflation_df['DATE'])
inflation_df['YEAR'] = inflation_df['DATE'].dt.year
inflation_df.rename(columns={"FPCPITOTLZGUSA": "CPI"}, inplace=True)

In [49]:
population_df = pd.read_csv('../data/ECONOMIC/POPTHM.csv')
population_df['DATE'] = pd.to_datetime(population_df['DATE'])
population_df.rename(columns={"POPTHM": "POP"}, inplace=True)

In [50]:
# Merge on date
ts = ts.merge(unemployment, how="left", on='DATE')
ts = ts.merge(population_df, how='left', on='DATE')
ts = ts.merge(inflation_df[['YEAR', 'CPI']], how='left', on='YEAR')

#ts.dropna(inplace=True)
# ts2.dropna(inplace=True)

### DON'T use murder rate, not sure how many people in each demopgraphic exist in total population

In [56]:
# use pop to get in family murder RATE  
ts['MURDERS_PER_100K'] = ts['Counts'] / (ts['POP'] / 100)

## More complicated Reg

In [76]:
# DON'T MODIFY ts HERE

# drop any rows that are not infamily murders
reg_data = ts[ts['IN_FAMILY'] == 1]

x = reg_data[["UNRATE", 'CPI', "WHITE_OFF", "MALE_OFF", "IN_RURAL", 'PERIOD', 'PERIOD_SQ']]
x = sm.add_constant(x)
y = ((reg_data["RATE_PER_100K"]))


model = sm.OLS(y, x).fit()

print(model.summary())

from statsmodels.iolib.summary2 import summary_col
res = summary_col(model)
print(res)

# save as image
# import matplotlib.pyplot as plt
# fig, ax = plt.subplots(figsize=(16, 8))
# summary = []
# model.summary(print_fn=lambda x: summary.append(x))
# summary = '\n'.join(summary)
# ax.text(0.01, 0.05, summary, fontfamily='monospace', fontsize=12)
# ax.axis('off')
# plt.tight_layout()
# plt.savefig('output.png', dpi=300, bbox_inches='tight')



                            OLS Regression Results                            
Dep. Variable:          RATE_PER_100K   R-squared:                       0.142
Model:                            OLS   Adj. R-squared:                  0.141
Method:                 Least Squares   F-statistic:                     109.9
Date:                Wed, 24 Apr 2024   Prob (F-statistic):          1.26e-128
Time:                        17:21:46   Log-Likelihood:                 14115.
No. Observations:                3988   AIC:                        -2.822e+04
Df Residuals:                    3981   BIC:                        -2.817e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0068      0.001      6.020      0.0

## Simple Reg : # of in family murders ~ inflation

In [9]:
# DON'T MODIFY ts HERE

# drop any rows that are now infamily murders
reg_data = ts[ts['IN_FAMILY'] == 1]

x = reg_data[['PERIOD', "CPI"]]
x = sm.add_constant(x)

y = ((reg_data["Counts"]))


sm.OLS(y, x).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Counts   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.051
Method:                 Least Squares   F-statistic:                     107.5
Date:                Wed, 24 Apr 2024   Prob (F-statistic):           3.33e-46
Time:                        16:10:27   Log-Likelihood:                -17040.
No. Observations:                3988   AIC:                         3.409e+04
Df Residuals:                    3985   BIC:                         3.410e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         25.0728      1.206     20.783      0.000      22.708      27.438
PERIOD        -0.0395      0.004     -9.860      0.000      -0.047      -0.032
CPI            0.0950      0.133      0.715      0.474      -0.165       0.355
==============================================================================
Omnibus:                      781.990   Durbin-Watson:                   0.851
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1352.148
Skew:                           1.276   Prob(JB):                    2.43e-294
Kurtosis:                       4.276   Cond. No.                         853.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""